## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-21-01-15-13 +0000,nyt,How Venezuela Went From U.S. Ally to Trump Target,https://www.nytimes.com/2025/12/19/us/politics...
1,2025-12-21-01-14-47 +0000,startribune,Ejection reveals a side of Timberwolves’ Chris...,https://www.startribune.com/ejection-reveals-a...
2,2025-12-21-01-09-01 +0000,nyt,U.S. Coast Guard Boards Tanker Carrying Venezu...,https://www.nytimes.com/2025/12/20/us/politics...
3,2025-12-21-01-03-23 +0000,missionlocal,"PG&E outage leaves 125,000 without power in Sa...",https://missionlocal.org/2025/12/sf-power-outa...
4,2025-12-21-01-00-00 +0000,wsj,Trump World Is Picking Sides in the Battle for...,https://www.wsj.com/business/media/trump-world...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,22
122,epstein,17
123,files,13
0,venezuela,8
21,oil,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
154,2025-12-20-04-35-07 +0000,nyt,Epstein Files Timeline: How the Trump Administ...,https://www.nytimes.com/2025/12/19/us/politics...,68
19,2025-12-20-23-45-00 +0000,nyt,Epstein Files Photos Disappear From Government...,https://www.nytimes.com/2025/12/20/us/politics...,64
156,2025-12-20-04-24-00 +0000,wsj,Days after President Trump moved to block stat...,https://www.wsj.com/articles/new-york-signs-ai...,60
181,2025-12-20-01-20-00 +0000,wsj,The Justice Department has released files rela...,https://www.wsj.com/politics/policy/justice-de...,57
10,2025-12-21-00-45-58 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...,57


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
154,68,2025-12-20-04-35-07 +0000,nyt,Epstein Files Timeline: How the Trump Administ...,https://www.nytimes.com/2025/12/19/us/politics...
156,38,2025-12-20-04-24-00 +0000,wsj,Days after President Trump moved to block stat...,https://www.wsj.com/articles/new-york-signs-ai...
10,35,2025-12-21-00-45-58 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...
66,35,2025-12-20-18-38-00 +0000,nypost,Elderly NYC woman found beaten to death near ‘...,https://nypost.com/2025/12/20/us-news/senior-c...
70,34,2025-12-20-18-21-29 +0000,nypost,US kills ‘at least 5’ ISIS thugs in Syria in r...,https://nypost.com/2025/12/20/us-news/us-shock...
82,31,2025-12-20-16-36-01 +0000,nypost,Former Hamas hostage warned leaders of rising ...,https://nypost.com/2025/12/20/world-news/forme...
36,30,2025-12-20-21-46-48 +0000,nypost,Russian envoy KO’s Trump administration propos...,https://nypost.com/2025/12/20/world-news/russi...
54,28,2025-12-20-19-43-37 +0000,nypost,Helen Garay Sanchez’s family speaks out for fi...,https://nypost.com/2025/12/20/us-news/helen-ga...
28,25,2025-12-20-22-31-17 +0000,nypost,Homeless hero who cracked Brown University and...,https://nypost.com/2025/12/20/us-news/homeless...
92,25,2025-12-20-15-32-25 +0000,nypost,Israel’s Benjamin Netanyahu expected to pitch ...,https://nypost.com/2025/12/20/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
